# Fine-Tuning for Rerank

Cohere's Rerank endpoint is a sophisticated semantic relevance scoring and ranking system that optimizes search results by evaluating the contextual relationship between queries and passages.

However, complex domains are special challenge due to their intricate terminology, context, and domain-specific knowledge requirements. These domains include legal documents, medical research papers, scientific literature, technical manuals, developer documentation, code, financial reports, and other fields that demand a deep understanding of specific jargon and intricate concepts. These domains often necessitate fine-tuning on custom data to ensure the models capture the nuances and expertise essential for accurate comprehension.

To understand the importance of domain-specific training, we will work with a code example utilizing a dataset in the legal domain. You'll see how fine-tuning can dramatically increase model accuracy.

## Overview

We'll do the following steps:
- **Step 1: Prepare the Dataset** - Download the dataset, select a subset, and prepare it for the Rerank endpoint.
- **Step 2: Assess the Pre-Trained Model** - Calculate the test accuracy of the pre-trained model.
- **Step 3: Fine-Tune the Model** - Kick off a fine-tuning job, and confirm when the model has completed training.
- **Step 4: Evaluate the Fine-Tuned Model** - Evaluate the fine-tuned model's performance on the test dataset.

## Setup

We'll start by installing the tools we'll neeed and then importing them.

In [1]:
! pip install cohere jsonlines datasets -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.0 MB/s eta 0:00:00


In [2]:
import os
import cohere
import json
import jsonlines
import pandas as pd
from sklearn.model_selection import train_test_split
from datasets import load_dataset

Fill in your Cohere API key in the next cell. To do this, begin by [signing up to Cohere](https://os.cohere.ai/) (for free!) if you haven't yet. Then get your API key [here](https://dashboard.cohere.com/api-keys).

In [11]:
# Paste your API key here. Remember to not share publicly
co = cohere.Client("COHERE_API_KEY")

## Step 1: Prepare the Dataset

We begin by downloading [the CaseHOLD dataset](https://huggingface.co/datasets/casehold/casehold) from Hugging Face. CaseHOLD is a multiple choice Q&A task consisting of legal decisions referencing other decisions as precedents, called a holding statement. It's a challenging task that demands specialized legal expertise to solve.

<img src="https://txt.cohere.com/content/images/2023/09/Untitled.png">

We define it as an [IterableDataset](https://huggingface.co/docs/datasets/en/about_mapstyle_vs_iterable) to load only a small fraction of examples at a time and avoid loading the entire dataset in memory.

In [5]:
iterable_dataset = load_dataset("casehold/casehold", split="train", streaming=True, trust_remote_code=True)

For this example, we'll use a subset of only 2000 data points, to be split across training, validation and test sets.

The data is stored in a Pandas DataFrame `df` with 5 columns:
- `"query"` - The search query or question (in the image above, this corresponds to the "citing text" or "prompt")
- `"docs"` - A list of five documents, where only one correctly answers the query (in the image above, all five "holding statements")
- `"label"` - The index of the document that correctly answers the query (in the example in the image above, would be "0", corresponding to holding statement 0)
- `"relevant_passages"` - The document that correctly answers the query
- `"hard_negatives"`- The four documents that don't correctly answer the query

In [6]:
# Size of data subset
num_examples = 2000

# Labels for columns containing the 5 documents in raw dataset file
all_labels = ["holding_" + s for s in ["0", "1", "2", "3", "4"]]

# Instantiate list containing the data
d = []

# Store each dataset entry as dictionary within Python list
for example in iterable_dataset.take(num_examples):
    selected_passage_idx = "holding_{}".format(example["label"])
    hard_negatives_idx = [x for x in all_labels if x != selected_passage_idx]
    d.append(
        {
            'query': example["citing_prompt"],
            'docs': [example.get(key) for key in all_labels],
            'label': int(example["label"]),
            'relevant_passages': [example[selected_passage_idx]],
            'hard_negatives': [example.get(key) for key in hard_negatives_idx]
        }
    )

# Convert list to Pandas dataframe, preview the dataframe
df = pd.DataFrame(d)
df.head()

,query,docs,label,relevant_passages,hard_negatives
0,"Drapeau’s cohorts, the cohort would be a “vict...",[holding that possession of a pipe bomb is a c...,0,[holding that possession of a pipe bomb is a c...,[holding that bank robbery by force and violen...
1,Colameta used customer information that he too...,[recognizing that even if a plaintiff claims c...,1,[holding that included among trade secrets emp...,[recognizing that even if a plaintiff claims c...
2,property tax sale. In reviewing section 6323(b...,[holding that where there is a conflict betwee...,4,[holding that a specific statutory provision p...,[holding that where there is a conflict betwee...
3,They also rely on Oswego Laborers’ Local 214 P...,[holding that plaintiff stated a 349 claim wh...,0,[holding that plaintiff stated a 349 claim wh...,[holding that plaintiff stated a claim for bre...
4,did not affect the defendant’s guideline range...,[holding that united states v booker 543 us 22...,3,[holding that united states v booker 543 us 22...,[holding that united states v booker 543 us 22...


We do a train-validation-test-split with a ratio of 75% for training (in `df_train`), 15% for validation (in `df_valid)`, and 10% for testing (in `df_test`).

In [7]:
# Set ratios for train-valid-test split (should sum to 1)
train_ratio = 0.75
valid_ratio = 0.15
test_ratio = 0.10

# Do train-validation-test split
df_train, df_val_test = train_test_split(df, test_size = 1 - train_ratio)
df_valid, df_test = train_test_split(df_val_test, test_size=test_ratio/(test_ratio + valid_ratio))

print('Size of training set:', len(df_train))
print('Size of validation set:', len(df_valid))
print('Size of teset set:', len(df_test))

Size of training set: 1500
Size of validation set: 300
Size of teset set: 200


## Step 2: Assess the Pre-Trained Model

We'll now check the test accuracy of the pre-trained model. The `get_prediction()` function looks at a test example and uses the pre-trained model to predict the index of the document that it believes correctly answers the query.

To get predictions, we'll use the [`rerank()` method](https://docs.cohere.com/reference/rerank-1) of the Cohere client and supply four arguments:
- `model` - Defaults to [`rerank-english-v2.0`](https://docs.cohere.com/docs/rerank-2), Cohere's pre-trained model for re-ranking English language documents
- `query` - The search query or question
- `documents` - List of documents to choose from
- `top_n` - Number of documents to return

In [8]:
# Predict index of document that corrrectly answers query
def get_prediction(item, model="rerank-english-v2.0"):
    response = co.rerank(model=model,
                         query=item["query"],
                         documents=item["docs"],
                         top_n=1)
    prediction = response.results[0].index
    return prediction

We apply this function to every row in the test set and save the predictions in new column `"baseline_prediction"`. Then, to calculate the test accuracy, we compare the predictions to the ground truth labels in the `"label"` column.

In [ ]:
# Calculate pre-trained model's test accuracy
df_test["baseline_prediction"] = df_test.apply(get_prediction, axis=1)
print("Baseline accuracy:", sum(df_test["baseline_prediction"] == df_test["label"])/len(df_test))

Baseline accuracy: 0.325


The model does better than random guessing (which would yield an accuracy of 25%), but we can do significantly better with fine-tuning.

## Step 3: Fine-Tune the Model

To prepare for fine-tuning with the Rerank endpoint, we'll need to convert the data to jsonl format, where each row is an example with three items:
- `"query"` - The search query or question
- `"relevant_passages"` - The document that correctly answers the query
- `"hard_negatives"`- The four documents that incorrectly answer the query

We do this separately for training and validation data. You can learn more about preparing the Rerank fine-tuning data in [the documentation](https://docs.cohere.com/docs/rerank-preparing-the-data).

In [9]:
# Arranges the data to suit Cohere's format
def create_rerank_ft_data(query, relevant_passages, hard_negatives):
    formatted_data = {
        "query": query,
        "relevant_passages": relevant_passages,
        "hard_negatives": hard_negatives
    }
    return formatted_data

# Creates jsonl file if does not already exist
def create_jsonl_from_list(file_name, df):
    path = f'{file_name}.jsonl'
    if not os.path.isfile(path):
        with open(path, 'w+') as file:
            for idx, row in df.iterrows():
                formatted_data = create_rerank_ft_data(row["query"], row["relevant_passages"], row["hard_negatives"])
                file.write(json.dumps(formatted_data) + '\n')
            file.close()

# Create training and validation jsonl files
create_jsonl_from_list("casehold_train", df_train)
create_jsonl_from_list("casehold_valid", df_valid)

Next, we preview the first couple lines of the training file.

In [ ]:
# List the first 2 items in the training jsonl file
with jsonlines.open('casehold_train.jsonl') as f:
    [print(line) for _, line in zip(range(2), f)]

{'query': 'other circuits which indicate that the failure to provide such notice is reversible error, as the parties may have engaged in a different trial strategy had they known of the jury\'s advisory capacity. See Thompson v. Parkes, 963 F.2d 885, 888 (6th Cir.1992) (examining the plain language of the rule and policy considerations of trial tactics and concluding "the rule requires that the court’s initiative in ordering a trial to an advisory jury must occur, and the parties be made aware of it, before the case is submitted”); Bereda v. Pickering Creek Indus. Park, Inc., 865 F.2d 49, 53 (3d Cir.1989) (requiring a district court to "notify both sides of a jury\'s advisory status no later than the time at which the jury selection has begun”); Pradier v. Elespuru, 641 F.2d 808, 811 (9th Cir.1981) (<HOLDING>); cf. Merex A.G. v. Fairchild Weston Sys.,', 'relevant_passages': ['recognizing that there are frequently significant tactical differences in presenting a case to a court as oppos

We kick off a fine-tuning job by navigating to the fine-tuning tab of the Dashboard. Under "Rerank", click on "Create a Rerank model".

<img src="https://files.readme.io/48dad78-cohere_dashboard.png">

Next, upload the .jsonl files you just created as the training and validation sets by clicking on the "TRAINING SET" and "VALIDATION SET" buttons. When ready, click on "Review data" to proceed to the next step.

<img src="https://files.readme.io/4522c16-rerank-review-data.png">

Then, you'll see a preview of how the model will ingest your data. If anything is wrong with the data, the page will also provide suggested changes to fix the training file. Otherwise, if everything looks good, you can proceed to the next step.

<img src="https://files.readme.io/3becc26-rerank-create-finetune.png">

Finally, you'll provide a nickname for your model. We used `casehold-rerank-ft` as the nickname for our model. This page also allows you to provide custom values for the hyperparameters used during training, but we'll keep them at the default values for now.

<img src="https://files.readme.io/ea79369-casehold-rerank-ft.png">

Once you have filled in a name, click on "Start training" to kick off the fine-tuning process. This will navigate you to a page where you can monitor the status of the model. A model that has finished fine-tuning will show the status as `READY`.

[image - need to take]

## Step 4: Evaluate the Fine-Tuned Model

Once the model has completed the fine-tuning process, it’s time to evaluate its performance.

Navigate to the API tab of the fine-tuned model. There, you'll see the model ID that you should use when calling `co.rerank()`.

[image - need to take]

In the following code, we calculate the test accuracy of the fine-tuned model. We use the same `get_prediction()` function as before, but now just need to pass in the fine-tuned model ID.

In [12]:
# Calculate fine-tuned model's test accuracy
df_test['ft_prediction'] = df_test.apply(get_prediction, model='48be5fa4-b16f-4fa4-baa7-a5023daa8af5-ft', axis=1)
print("Fine-tune accuracy:", sum(df_test["ft_prediction"] == df_test["label"])/len(df_test))

Fine-tune accuracy: 0.675


This is a meaningful increase in accuracy, and we could improve performance even more by increasing the size of the training data!